In [1]:
import pandas as pd
import numpy as np

In [2]:
# Lecture du fichier *.txt
data = pd.read_csv('fredmq.txt', sep=',').set_index(keys='Date')
# Récupération des types de transformations dans un dictionnaire
transformations = dict(data.loc['Transform']) 
# Abandon de la ligne des transformations
data.drop(labels='Transform', inplace=True)
# Ajustement de l'indice du DataFrame au format datetime
data.index = pd.to_datetime(arg=data.index)
# Abandon des colonnes ayant un nombre de NaN's >= 30
data.dropna(thresh=len(data) - 30, axis=1, inplace=True)
# Abandon des NaN's restants
data.dropna(axis=0, inplace=True)

In [3]:
# Création du DataFrame stockant les séries transformées
transformed_data = pd.DataFrame()
# Transformation des séries selon les codes fournis par McCracken et Ng
for col in data.columns:
    # Récupération du code de transformation
    code = transformations[col]
    # Récupération de la série individuelle
    subset = data[col]
    # Application des transformées selon le code correspondant
    if code == 1:
        # Aucune transformation à appliquer
        temp = subset
    elif code == 2:
        # Différence première
        temp = subset.diff(periods=1)
    elif code == 3:
        # Différence seconde 
        temp = subset.diff(periods=1).diff(periods=1)
    elif code == 4:
        # Transformée en log
        temp = np.log(subset)
    elif code == 5:
        # Différence première du log
        temp = np.log(subset).diff(periods=1)
    elif code == 6:
        # Différence seconde du log 
        temp = np.log(subset).diff(periods=1).diff(periods=1)
    elif code == 7:
        # Différence première de la variation relative
        temp = subset.pct_change().diff(periods=1)
    # Aggrégation des séries transformées
    transformed_data = pd.concat([transformed_data, temp], axis=1)